# Q1 Project Notebook

In [1]:
#imports
import pandas as pd
import networkx
import geopandas as gpd
from shapely import wkt
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import networkx as nx
import pandas as pd
import geopandas as gpd
from shapely import wkt
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
import branca.colormap as cm

In [2]:
#load in datasets
wind_data = pd.read_excel('src_wings_meteorology_windspeed_snapshot_2023_08_02.xlsx')
alert_data = pd.read_excel('src_wings_meteorology_station_summary_snapshot_2023_08_02.xlsx')
station_data = pd.read_excel('gis_weatherstation_shape_2024_10_04.xlsx')
conductor_data = pd.read_csv('dev_wings_agg_span_2024_01_01.csv')
vri_data = pd.read_csv('src_vri_snapshot_2024_03_20.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'dev_wings_agg_span_2024_01_01.csv'

# Task 1 EDA and Merging of Wind, Alert, Station Data

In [ ]:
#drop duplicates in wind_data
cleaned_station_data = station_data.drop_duplicates(subset=['weatherstationcode'], keep='first')

In [ ]:
alert_data

## Merging Datasets

In [ ]:
#first merge station data and alert_data to get the alert data for each station
station_alert_data = pd.merge(cleaned_station_data,alert_data[['station','vri','alert']], left_on = 'weatherstationcode', right_on = 'station').drop('station',axis = 1)

In [ ]:
wind_station_alert_data = pd.merge(wind_data, station_alert_data, 
                     left_on='station', 
                     right_on='weatherstationcode', 
                     how='left')

wind_station_alert_data

In [ ]:
wind_speed_stats = wind_station_alert_data.groupby('station')['wind_speed'].agg(['mean', 'min', 'max'])
wind_speed_stats

## EDA

In [ ]:
# Historical Wind Speed Distribution
plt.figure(figsize=(12,10))
sns.histplot(data=wind_station_alert_data, x='wind_speed', kde=True, bins=30, color='skyblue')
plt.title('Historical Wind Speed Distribution')
plt.xlabel('Wind Speed (km/h)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
#Distribution of Alert Speeds
plt.figure(figsize=(10, 6))
sns.histplot(data=wind_station_alert_data, x='alert', bins=5, kde=True, color='skyblue')

# Add titles and labels
plt.title('Distribution of Alert Speeds')
plt.xlabel('Alert Speed (km/h)')
plt.ylabel('Frequency')

# Adjust x-axis to show only the range of alert speeds
plt.xlim(wind_station_alert_data['alert'].min() - 5, wind_station_alert_data['alert'].max() + 5)

# Display the histogram
plt.show()

In [ ]:
wind_station_alert_data['geometry'] = wind_station_alert_data.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
wind_station_alert_data = gpd.GeoDataFrame(wind_station_alert_data, geometry='geometry')
wind_station_alert_data.crs = 'EPSG:4326'

wind_station_alert_data = wind_station_alert_data.to_crs(epsg=4326)

In [ ]:
# Convert 'date' to datetime
wind_station_alert_data['date'] = pd.to_datetime(wind_station_alert_data['date'])

for station in wind_station_alert_data['station'].unique():
    station_data = wind_station_alert_data[wind_station_alert_data['station'] == station]
    plt.figure()
    plt.plot(station_data['date'], station_data['wind_speed'], label=f"Wind Speed at {station}")
    plt.title(f"Wind Speed Over Time at {station}")
    plt.xlabel("Date")
    plt.ylabel("Wind Speed")
    plt.legend()
    plt.show()


In [ ]:
# Create a base map
m = folium.Map(location=[gdf_weatherstation.geometry.y.mean(), gdf_weatherstation.geometry.x.mean()], zoom_start=10)

# Add markers for each station with a popup showing key metrics
for _, row in gdf_weatherstation_merged.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Station: {row['station']}, Wind Speed: {row['wind_speed']}, Max Gust: {row['max_gust']}"
    ).add_to(m)

# Display the map
display(m)

# Task 2 Calculating PSPS Probability of Weather Stations and displaying results
- 1. Calculating PSPS Probability for each Weather Station 
- 2. Visualizing Weather Station wind speed distributions against its wind speed threshold, and displaying its calculated PSPS Probability

In [ ]:
weather_station_psps_prob_data = (
    wind_station_alert_data.assign(exceeds_alert=lambda x: (x['wind_speed'] > x['alert']))  # Create a boolean column
      .groupby('weatherstationcode')
      .agg(total_records=('exceeds_alert', 'size'),  # Total records per station
           exceeds_count=('exceeds_alert', 'sum'))   # Count where wind_speed > alert
      .assign(percentage_of_PSPS_shutdown=lambda  x: (x['exceeds_count'] / x['total_records']))
      [['percentage_of_PSPS_shutdown']]  # Select only the relevant column
)

weather_station_psps_prob_data.sort_values(by = 'percentage_of_PSPS_shutdown', ascending = False)

In [ ]:
merged_df = pd.merge(wind_station_alert_data, weather_station_psps_prob_data, on='weatherstationcode')

In [ ]:
grouped = merged_df.groupby('station')

In [ ]:
# Plot settings
stations_per_batch = 9  # Number of stations per figure (e.g., a grid of 3x3 subplots)
num_stations = len(grouped)
num_batches = (num_stations // stations_per_batch) + (num_stations % stations_per_batch > 0)

# Loop through batches
for batch in range(num_batches):
    start_idx = batch * stations_per_batch
    end_idx = min(start_idx + stations_per_batch, num_stations)
    
    # Create subplots for this batch
    fig, axes = plt.subplots(
        nrows=3,
        ncols=3,
        figsize=(15, 12),
        sharey=True,
        tight_layout=True
    )
    axes = axes.flatten()  # Flatten axes for easier iteration
    
    for ax_idx, (station_name, group) in enumerate(list(grouped)[start_idx:end_idx]):
        ax = axes[ax_idx]
        
        # Plot histogram of wind speeds
        ax.hist(group['wind_speed'], bins=10, alpha=0.7, color='skyblue', edgecolor='black', label='Wind Speed')
        
        # Add vertical line for wind speed threshold
        threshold = group['alert'].iloc[0]
        ax.axvline(threshold, color='red', linestyle='dashed', linewidth=2, label='Threshold')
        
        # Add title with PSPS probability
        psps_probability = group['percentage_of_PSPS_shutdown'].iloc[0]
        ax.set_title(f'{station_name}\nPSPS Probability: {psps_probability:.2f}')
        
        # Add labels and legend
        ax.set_xlabel('Wind Speed (m/s)')
        ax.set_ylabel('Frequency')
        ax.legend()
    
    # Hide unused subplots in the grid
    for ax in axes[len(list(grouped)[start_idx:end_idx]):]:
        ax.axis('off')
    
    # Save or show the plot for this batch
    plt.tight_layout()
    plt.savefig(f'weather_station_batch_{batch + 1}.png')  # Save each batch as a separate file
    plt.show()

# Task 3 Merging weather station data to conductor spans and displaying PSPS Probabilities across all 3 layers geospatially
- 1) Merge weather stations to VRI polygons
- 2) Merge VRI polgyons to Conductor Spans
- 3) Displaying PSPS Probabilities across all 3 layers geospatially

In [ ]:
#merging weather station + alert data and vri data
station_vri_data = station_alert_data.merge(vri_data, left_on="weatherstationcode", right_on="anemometercode", how="inner")
station_vri_data = pd.merge(station_vri_data, weather_station_psps_prob_data, on='weatherstationcode')
def rename_columns(col):
    if col.endswith('_x'):
        return col[:-2] + '_station'
    elif col.endswith('_y'):
        return col[:-2] + '_vri'
    return col

# Rename the columns
station_vri_data = station_vri_data.rename(columns=rename_columns)

In [ ]:
station_vri_data

In [ ]:
#weather station and vri data + conductor span data geospatial span
import pandas as pd
import geopandas as gpd
from shapely import wkt


# Convert WKT strings to geometry objects for conductor data
conductor_data['geometry'] = conductor_data['shape'].apply(wkt.loads)

# Create GeoDataFrame from conductor data
gdf1 = gpd.GeoDataFrame(conductor_data, geometry='geometry', crs=f"EPSG:{conductor_data['shape_srid'].iloc[0]}")

# convert WKT string to geometry objects for conductor data
station_vri_data['geometry'] = station_vri_data['shape_vri'].apply(wkt.loads)
gdf2 = gpd.GeoDataFrame(station_vri_data, geometry='geometry', crs=f"EPSG:{station_vri_data['shape_srid_vri'].iloc[0]}")

# Ensure both GeoDataFrames have the same CRS
if gdf1.crs != gdf2.crs:
    gdf2 = gdf2.to_crs(gdf1.crs)

# Perform the spatial join
joined_gdf = gpd.sjoin(gdf1, gdf2, how="left", predicate="intersects")

# Post-processing steps
# Rename columns from VRI data to avoid confusion
columns_to_rename = [col for col in joined_gdf.columns if col in gdf2.columns and col != 'geometry']
joined_gdf = joined_gdf.rename(columns={col: f"{col}_vri" for col in columns_to_rename})
joined_gdf.columns = joined_gdf.columns.str.replace('vri_vri', 'vri')
# Drop unnecessary columns
columns_to_drop = ['index_right'] 
joined_gdf = joined_gdf.drop(columns=columns_to_drop, errors='ignore')

# Reset index if needed
joined_gdf = joined_gdf.reset_index(drop=True)

In [ ]:
joined_gdf = pd.merge(joined_gdf, weather_station_psps_prob_data, left_on='weatherstationcode_vri', right_on ='weatherstationcode')

In [ ]:
station_vri_data[['geometry']]

In [ ]:
#plotting weather stations and psps probabilites

In [ ]:
station_psps_data = pd.merge(cleaned_station_data,weather_station_psps_prob_data, on = "weatherstationcode")

In [ ]:
center_lat = station_psps_data['latitude'].mean()
center_lon = station_psps_data['longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

In [ ]:
for _, station in station_psps_data.iterrows():
    folium.Marker(
        location=[station['latitude'], station['longitude']],
        popup=f"Station: {station['weatherstationname']}<br>PSPS Probability: {station['percentage_of_PSPS_shutdown']:.2f}",
        tooltip=station['weatherstationname']
    ).add_to(m)

# Save the map
m.save("weather_stations_map.html")

In [ ]:
vri_data_psps_prob = pd.merge(vri_data, weather_station_psps_prob_data, right_on='weatherstationcode',left_on = "anemometercode")
# Assuming your DataFrame is named 'df'
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(vri_data_psps_prob, geometry=gpd.GeoSeries.from_wkt(vri_data_psps_prob['shape']))
gdf.set_crs(epsg=4326, inplace=True)
m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=6)

# Function to style each featureSW
def style_function(feature):
    return {
        'fillColor': '#ffaf00',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.7,
    }

# Add GeoJson layer
folium.GeoJson(
    gdf,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['name', 'percentage_of_PSPS_shutdown', 'hftd'], aliases=['Name', 'Percentage_PSPS_Shutdown', 'HFTD']),
    popup=folium.GeoJsonPopup(fields=['name', 'vri_risk', 'hftd', 'gust_max'], aliases=['Name', 'VRI Risk', 'HFTD', 'Max Gust'])
).add_to(m)

# Save the map
m.save("vri_shapes_pspsprob.html")


In [ ]:
colormap = cm.LinearColormap(
    colors=['green', 'yellow', 'red'],
    vmin=gdf['percentage_of_PSPS_shutdown'].min(),
    vmax=gdf['percentage_of_PSPS_shutdown'].max(),
    caption='PSPS Shutdown Probability'
)

# Modify the style function to use the colormap
def style_function(feature):
    psps_value = feature['properties']['percentage_of_PSPS_shutdown']
    return {
        'fillColor': colormap(psps_value),
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.7,
    }

# Create the map
m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], 
               zoom_start=6)

# Add GeoJson layer with the new style function
folium.GeoJson(
    gdf,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['name', 'percentage_of_PSPS_shutdown', 'hftd'], 
        aliases=['Name', 'Percentage_PSPS_Shutdown', 'HFTD']
    ),
    popup=folium.GeoJsonPopup(
        fields=['name', 'vri_risk', 'hftd', 'gust_max'], 
        aliases=['Name', 'VRI Risk', 'HFTD', 'Max Gust']
    )
).add_to(m)

# Add the colormap to the map
colormap.add_to(m)

# Save the map
m.save("vri_shapes_pspsprob2.html")

# Task 4 Creating graph network object of spans to trace upstream/downtream the grid; collect list of weather stations that could cause a shut-off to any given span 

In [ ]:
#drop duplicate station data
station_data = station_data.drop_duplicates(subset=['weatherstationcode'], keep='first')
#merge alert data to station data so for each station have alert threshold
alert_station_data = pd.merge(station_data,alert_data[['station','vri','alert']], left_on = 'weatherstationcode', right_on = 'station').drop('station',axis = 1)

station_vri_data = alert_station_data.merge(vri_data, left_on="weatherstationcode", right_on="anemometercode", how="inner")

# Define a function to rename columns
def rename_columns(col):
    if col.endswith('_x'):
        return col[:-2] + '_station'
    elif col.endswith('_y'):
        return col[:-2] + '_vri'
    return col

# Rename the columns
station_vri_data = station_vri_data.rename(columns=rename_columns)


In [ ]:
#weather station and vri data + conductor span data geospatial span
import pandas as pd
import geopandas as gpd
from shapely import wkt

# Assuming df1 is your Conductor span DataFrame and df2 is your VRI polygon DataFrame

# Convert WKT strings to geometry objects for df1
conductor_data['geometry'] = conductor_data['shape'].apply(wkt.loads)

# Create GeoDataFrame from df1
gdf1 = gpd.GeoDataFrame(conductor_data, geometry='geometry', crs=f"EPSG:{conductor_data['shape_srid'].iloc[0]}")

# Do the same for df2 if necessary
station_vri_data['geometry'] = station_vri_data['shape_vri'].apply(wkt.loads)
gdf2 = gpd.GeoDataFrame(station_vri_data, geometry='geometry', crs=f"EPSG:{station_vri_data['shape_srid_vri'].iloc[0]}")

# Ensure both GeoDataFrames have the same CRS
if gdf1.crs != gdf2.crs:
    gdf2 = gdf2.to_crs(gdf1.crs)

# Perform the spatial join
joined_gdf = gpd.sjoin(gdf1, gdf2, how="left", predicate="intersects")

# Post-processing steps
# Rename columns from VRI data to avoid confusion
columns_to_rename = [col for col in joined_gdf.columns if col in gdf2.columns and col != 'geometry']
joined_gdf = joined_gdf.rename(columns={col: f"{col}_vri" for col in columns_to_rename})

# Drop unnecessary columns
columns_to_drop = ['index_right']  # Add any other columns you don't need
joined_gdf = joined_gdf.drop(columns=columns_to_drop, errors='ignore')

# Reset index if needed
joined_gdf = joined_gdf.reset_index(drop=True)

In [ ]:
G = nx.DiGraph()

for _, row in conductor_data.iterrows():
    if pd.notnull(row['upstream_span_id']):
        upstream_id = row['upstream_span_id']
        current_id = row['globalid']
        G.add_edge(upstream_id, current_id)

In [ ]:
joined_gdf.columns.tolist()

In [ ]:
new_joined_gdf = joined_gdf.merge(weather_station_psps_prob_data, left_on = 'station',right_on = "weatherstationcode",how = "left")

In [ ]:
new_joined_gdf = new_joined_gdf[(new_joined_gdf['percentage_of_PSPS_shutdown'] > 0)]
new_joined_gdf = new_joined_gdf[['globalid','upstream_span_id', 'weatherstationcode_vri','parent_feederid']].drop_duplicates(subset = ['upstream_span_id','weatherstationcode_vri'])
new_joined_gdf.dropna(inplace = True)
#new_joined_gdf.dropna(inplace = True, subset = ['upstream_span_id', 'station'])

In [ ]:
grouped_feederid_conductor_df= {group: data for group, data in new_joined_gdf.groupby('parent_feederid')}

In [ ]:
def trace_spans(G, span_id):
    """Get all upstream and downstream spans of a given span."""
    # downstream_spans = list(nx.descendants(G, span_id))
    upstream_spans = list(nx.ancestors(G, span_id))
    return upstream_spans

def get_weather_station_associations(impacted_spans):
    unique_spans = impacted_spans['upstream_span_id'].unique()
    span_weather_stations = {}

    for span_id in unique_spans:
        traced_spans = trace_spans(G, span_id)
        
        traced_spans.append(span_id)
        
        associated_stations = impacted_spans[
            impacted_spans['upstream_span_id'].isin(traced_spans)
        ]['weatherstationcode_vri'].unique()
        
        span_weather_stations[span_id] = list(associated_stations)

    output_df = pd.DataFrame(
        [(span, stations) for span, stations in span_weather_stations.items()],
        columns=['upstream_span_id', 'associated_station']
    )

    return output_df

In [ ]:

# Step 2: For each group, create a directed graph and trace upstream spans
def process_group(group_data):
    G = nx.DiGraph()
    # Add edges based on upstream and downstream structure IDs
    for _, row in group_data.iterrows():
        if pd.notnull(row['upstream_span_id']):
            upstream_id = row['upstream_span_id']
            current_id = row['globalid']
            G.add_edge(upstream_id, current_id)

    # Apply the function to populate the new column with upstream spans
    
    return get_weather_station_associations(group_data)

# Step 3: Apply the process to each group and concatenate the results
processed_groups = [process_group(group_data) for group_data in grouped_feederid_conductor_df.values()]
final_df = pd.concat(processed_groups)

# Step 4: Display the final DataFrame with the new column
print(final_df)

In [ ]:
final_df[final_df['upstream_span_id'] == "{85FA1DBE-DDAE-43D8-BBFA-CFD8C0D65678}"]

In [ ]:
final_df

In [ ]:

span_station_exploded = final_df.explode('associated_station').rename(columns={'associated_station': 'station'})

merged = span_station_exploded.merge(weather_station_psps_prob_data, left_on='station', right_on ="weatherstationcode",how='left')

def calculate_union_probability(group):
    """Calculate union probability for a span's associated stations."""
    probs = group['percentage_of_PSPS_shutdown'].dropna()
    if len(probs) == 0:
        return np.nan
    union_prob = 1 - np.prod(1 - probs)
    return union_prob

psps_probabilities = merged.groupby('upstream_span_id').apply(calculate_union_probability).reset_index()
psps_probabilities.columns = ['span_id', 'psps_probability']


In [ ]:
psps_probabilities

In [ ]:
psps_probabilities[psps_probabilities['span_id'] == "{FFF48154-5787-48BA-87CF-7C07CDAA3C75}"]

In [ ]:
joined_gdf[joined_gdf['upstream_span_id'] == "{0000602C-82D9-445A-B1AF-81BDFB0EEC14}"]

In [ ]:
joined_gdf_psps_probs = joined_gdf.merge(psps_probabilities,left_on = 'upstream_span_id',right_on ="span_id",how = "right" )

In [ ]:
joined_gdf_psps_probs[['upstream_span_id',"span_id"]]

In [ ]:
joined_gdf_psps_probs['upstream_span_id'].duplicated().sum()

In [ ]:
##########################

# Task 5 Computing PSPS Probability of every conductor span

In [ ]:
psps_probabilities

# Task 6 Estimate expected customers that'll be shut-off over the next 10 years at the span/segment/circuit granularity

### Expected Value approach
Say for example your Probability is 5% on any single high fire day captured in the station data. And then you can calculate the  average days per year that the station data has collected during these high fire days. Assume it's  13 days per year. Then 13 x 0.05 = 0.65 can be assessed as the expected annual rate of shut-offs for that span. And then 0.65 * 10 = 6.5 shut-offs expected over 10 years for the customers associated to that span. And then you could aggregated that to the feederid level.
That is one way to do it. 

In [ ]:
wind_data['wind_speed'].isna().sum()

In [ ]:
wind_data_cleaned = wind_data.dropna(subset = ['wind_speed'])

In [ ]:
wind_data_cleaned.groupby('station').size().unique() #each station has 179 entires 

In [ ]:
# Convert date to datetime and extract year
wind_data_cleaned['year'] = pd.to_datetime(wind_data_cleaned['date']).dt.year

# Group by year and station
grouped_df = wind_data_cleaned.groupby(['year', 'station']).size().reset_index(name = "Annual_high_fire_days")
grouped_df

In [ ]:
grouped_df[grouped_df['station'] == 'AMO']

In [ ]:
average_high_fire_days_per_station = grouped_df.groupby('station')['Annual_high_fire_days'].mean().reset_index(name='Average_Annual_high_fire_days')
average_high_fire_days_per_station.reset_index()

In [ ]:
weather_station_psps_prob_data

In [ ]:
#distribution of weather station psps shutdown percentage
plt.figure(figsize=(10, 6))
sns.kdeplot(data=weather_station_psps_prob_data['percentage_of_PSPS_shutdown'], fill=True)

# Customize the plot
plt.title('Distribution of PSPS Shutdown Probabilities')
plt.xlabel('Probability of PSPS Shutdown')
plt.ylabel('Density')

# Add a rug plot to show the actual data points
sns.rugplot(data=weather_station_psps_prob_data['percentage_of_PSPS_shutdown'], color='red', height=0.1)

plt.show()

In [ ]:
import seaborn as sns
# Create the histogram
plt.figure(figsize=(12, 6))
sns.histplot(data=weather_station_psps_prob_data, x='percentage_of_PSPS_shutdown', bins=20, kde=True)

# Customize the plot
plt.title('Distribution of PSPS Shutdown Probabilities', fontsize=16)
plt.xlabel('Probability of PSPS Shutdown', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

# Add a vertical line for the mean
mean_probability = weather_station_psps_prob_data['percentage_of_PSPS_shutdown'].mean()
plt.axvline(mean_probability, color='red', linestyle='dashed', linewidth=2, label=f'Mean: {mean_probability:.4f}')

# Add legend
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
test = pd.merge(average_high_fire_days_per_station,weather_station_psps_prob_data, left_on = "station", right_on = "weatherstationcode")
test

In [ ]:
test = test[['station','Average_Annual_high_fire_days','percentage_of_PSPS_shutdown']]
test['Annual_number_of_shutoffs_10years'] = (test['Average_Annual_high_fire_days'] * test['percentage_of_PSPS_shutdown'] * 10)
test

In [ ]:
shutoff_rates_dict = test.set_index('station')['Annual_number_of_shutoffs_10years'].to_dict()

In [ ]:
test2 = joined_gdf_psps_probs.merge(final_df,on = "upstream_span_id", how = "left")

In [ ]:
annual_rate_shutoff_dict = test.set_index('station')['Annual_number_of_shutoffs_10years'].to_dict()

In [ ]:
annual_rate_shutoff_dict

In [ ]:
def process_weather_station_annual_shutoff_rates(df, shutoff_rates_dict):
    def get_annual_shutoff_rates(station_codes):
        probabilities = [shutoff_rates_dict.get(code, 0) for code in station_codes]
        return probabilities
    
    def calculate_total_shutoffs(shutoffs):
        if not shutoffs:
            return 0
        shutoffs = [x for x in shutoffs if x != 0]
        total_prob = np.sum(shutoffs)
        return total_prob
    
    df['annual_shutoff_counts_10years'] = df['associated_station'].apply(get_annual_shutoff_rates)
    df['total_number_shutoff_10years'] = df['annual_shutoff_counts_10years'].apply(calculate_total_shutoffs)
    
    return df

In [ ]:
output.columns.tolist()

In [ ]:
output = process_weather_station_annual_shutoff_rates(test2,shutoff_rates_dict)
output['total_number_customers_affected_ten_years'] = output['total_number_shutoff_10years'] * (output['downstream_cust_total'] + output['cust_total'])
output

In [ ]:
output[['globalid','associated_station','upstream_struct_id','downstream_struct_id','downstream_cust_total','cust_total','annual_shutoff_counts_10years','total_number_shutoff_10years']]

In [ ]:
# Create distribution plot of total number of shutoffs
plt.figure(figsize=(12, 6))
sns.histplot(data=output, x='total_number_shutoff_10years', bins=50)
plt.title('Distribution of Total Number of Shutoffs Over 10 Years (Span Level)')
plt.xlabel('Number of Shutoffs')
plt.ylabel('Frequency')

# Create boxplot to show the distribution of shutoffs by number of associated stations
plt.figure(figsize=(12, 6))
output['num_stations'] = output['associated_station'].str.len()
sns.boxplot(x='num_stations', y='total_number_shutoff_10years', data=output)
plt.title('Distribution of Shutoffs by Number of Associated Stations')
plt.xlabel('Number of Associated Stations')
plt.ylabel('Total Number of Shutoffs (10 Years)')

# Create scatter plot of customers affected vs shutoffs
plt.figure(figsize=(12, 6))
sns.scatterplot(data=output, 
                x='total_number_shutoff_10years', 
                y='cust_total',
                alpha=0.5)
plt.title('Relationship Between Number of Shutoffs and Customers Affected')
plt.xlabel('Total Number of Shutoffs (10 Years)')
plt.ylabel('Number of Customers')

# Task 6 Estimate expected customers that'll be shut-off over the next 10 years at the span/segment/circuit granularity
#### segment level = upstreamardfacilityid
#### circuit = parent_feederid
#### span = globalid

# Circuit Level (feederid)

In [ ]:
aggregated_feederid_df = output.groupby('feederid', as_index=False).agg(
    total_number_shutoffs_feederid = ('total_number_shutoff_10years','sum'),
    total_customers = ('cust_total','sum'),
    total_downstream_customers = ('downstream_cust_total','sum'),
    total_number_affected_customers_10years = ('total_number_customers_affected_ten_years','sum')
)
aggregated_feederid_df

In [ ]:
# Create the histogram
plt.figure(figsize=(12, 6))
sns.histplot(data=aggregated_feederid_df, x='total_number_affected_customers_10years', bins=50)

# Customize the plot
plt.title('Distribution of Total Number of Customers Affected Over 10 Years (Span Level)', fontsize=14)
plt.xlabel('Number of Customers Affected', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

# Add grid for better readability
plt.grid(True, alpha=0.3)

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

# Span Level

In [ ]:
span_out = output[['globalid','total_number_shutoff_10years','cust_total','downstream_cust_total','total_number_customers_affected_ten_years']]
span_out

In [ ]:
conductor_data[conductor_data['globalid'] == "{E335ABFD-E9B0-497C-A182-837D25ED6FE4}"]['cust_total']

In [ ]:


# Create the histogram
plt.figure(figsize=(12, 6))
sns.histplot(data=span_out, x='total_number_customers_affected_ten_years', bins=10)

# Customize the plot
plt.title('Distribution of Total Number of Customers Affected Over 10 Years (Span Level)', fontsize=14)
plt.xlabel('Number of Customers Affected', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

# Add grid for better readability
plt.grid(True, alpha=0.3)

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

# Segment Level


In [ ]:
aggregated_feederid_df = output.groupby('upstreamardfacilityid', as_index=False).agg(
    total_number_shutoffs_feederid = ('total_number_shutoff_10years','sum'),
    total_customers = ('cust_total','sum'),
    total_downstream_customers = ('downstream_cust_total','sum'),
    total_number_affected_customers_10years = ('total_number_customers_affected_ten_years','sum')
)
aggregated_feederid_df